In [1]:
import torch

In [3]:
from deep_learning.models.feature_extractors.resnet import Resnet50

model= Resnet50()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

random_input = torch.randn(200, 3, 256, 256).to(device)

model.eval()
with torch.no_grad():
    features = model(random_input)

print(features.shape)

cuda
torch.Size([200, 2048, 1, 1])


In [ ]:
from deep_learning.models.feature_extractors.efficientnet import EfficientNetB1

model= EfficientNetB1()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

random_input = torch.randn(200, 3, 256, 256).to(device)

model.eval()
with torch.no_grad():
    features = model(random_input)

print(features.shape)

c:\Users\user\.julia\conda\3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\.julia\conda\3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B1_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


cuda
torch.Size([200, 1280, 1, 1])


In [3]:
import torch
import os
from deep_learning.feature_extraction import extract_features
from deep_learning.models.transforms.transforms import transform_resnet
from deep_learning.models.feature_extractors.resnet import Resnet50
from deep_learning.loaders.image_batch_loader import load_batch_from_dir
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=Resnet50().to(device)

features=extract_features(model,transform_resnet,"F:/test/0005f7aaab2800f6170c399693a96917/tiles")
torch.save(features,"F:/test/0005f7aaab2800f6170c399693a96917.pt")


c:\Users\user\.julia\conda\3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\.julia\conda\3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
import pandas as pd

# Example DataFrame (assuming it’s loaded from a CSV)
df = pd.DataFrame({
    "feature_path": ["features/img1.npy", "features/img2.npy", "features/img3.npy"],
    "label": [0, 1, 0]
})
df["feature_path"][0]

'features/img1.npy'

In [4]:
from deep_learning.loaders.feature_dataset import FeatureDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

df=pd.read_csv("F:/data/small.csv")
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print("Train size:", len(train_df))
print("Test size:", len(test_df))

dataset=FeatureDataset(train_df,"F:/extracted_features")

train_loader = DataLoader(dataset, shuffle=True)

print(f"Train batches: {len(train_loader)}")

for batch in train_loader:
    x_batch, y_batch = batch
    x_batch=x_batch.squeeze()
    print("Batch X shape:", x_batch.shape)  # Expected: (32, D)
    print("Batch Y shape:", y_batch.squeeze().shape)  # Expected: (32,)
    break  # Only one batch

Train size: 760
Test size: 191
Train batches: 760
Batch X shape: torch.Size([193, 2048])
Batch Y shape: torch.Size([])


In [5]:
from deep_learning.models.attention_core.clam import Clam
import torch
from torchinfo import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=Clam(
    feature_vector_length=2048,
    dropout=0.1,
    k_sample=8,
    n_classes=5,
    subtyping=True
)

model.to(device)
summary(model, input_size=(100,2048))

Layer (type:depth-idx)                   Output Shape              Param #
Clam                                     [1, 5]                    5,130
├─Sequential: 1-1                        [100, 5]                  --
│    └─Linear: 2-1                       [100, 512]                1,049,088
│    └─ReLU: 2-2                         [100, 512]                --
│    └─Dropout: 2-3                      [100, 512]                --
│    └─Attn_Net_Gated: 2-4               [100, 5]                  --
│    │    └─Sequential: 3-1              [100, 256]                131,328
│    │    └─Sequential: 3-2              [100, 256]                131,328
│    │    └─Linear: 3-3                  [100, 5]                  1,285
├─ModuleList: 1-2                        --                        --
│    └─Linear: 2-5                       [1]                       513
│    └─Linear: 2-6                       [1]                       513
│    └─Linear: 2-7                       [1]                

In [16]:
for batch in train_loader:
    x_batch, y_batch = batch
    x_batch=x_batch.squeeze()
    x_batch.to(device)
    logits, Y_prob, Y_hat, A_raw, results_dict=model(x_batch)
    print(logits)
    break  # Only one batch

tensor([[ 0.0363, -0.0058,  0.0846, -0.0593,  0.0797]], device='cuda:0',
       grad_fn=<CopySlices>)


In [1]:
from deep_learning.loaders.feature_dataset import FeatureDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from deep_learning.train.train import train
from deep_learning.models.attention_core.clam import Clam

df=pd.read_csv("F:/data/small.csv")
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

model=Clam(
    feature_vector_length=2048,
    dropout=0.1,
    k_sample=8,
    n_classes=6,
    subtyping=False
)



train(model,train_df,"F:/extracted_features",3)



batch 19, loss: 2.6146, instance_loss: 0.7178, weighted_loss: 1.6662, label: 2, bag_size: 198
batch 39, loss: 1.7910, instance_loss: 0.6853, weighted_loss: 1.2382, label: 5, bag_size: 145
batch 59, loss: 2.4337, instance_loss: 0.7029, weighted_loss: 1.5683, label: 3, bag_size: 128
batch 79, loss: 1.0695, instance_loss: 0.5800, weighted_loss: 0.8247, label: 1, bag_size: 395
batch 99, loss: 3.1832, instance_loss: 0.6473, weighted_loss: 1.9152, label: 2, bag_size: 184
batch 119, loss: 1.7686, instance_loss: 0.5912, weighted_loss: 1.1799, label: 1, bag_size: 246
batch 139, loss: 2.1358, instance_loss: 0.6433, weighted_loss: 1.3895, label: 3, bag_size: 167
batch 159, loss: 1.2676, instance_loss: 0.3248, weighted_loss: 0.7962, label: 0, bag_size: 236
batch 179, loss: 2.4001, instance_loss: 0.8914, weighted_loss: 1.6457, label: 5, bag_size: 134
batch 199, loss: 2.4610, instance_loss: 0.6065, weighted_loss: 1.5337, label: 2, bag_size: 225
batch 219, loss: 1.2204, instance_loss: 0.5954, weigh